In [1]:
# import dependencies
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import pymongo
import re as re

# Output File (CSV)
output_data_file = "csv_outputs/craigslist_tools.csv"

In [2]:
pg_num = 1
pg_cnt = 0
per_pg = 120
query = 'tools'.replace(' ', '+')
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://toronto.craigslist.org/search/sss?query='+ str(query) + '&sort=rel&s=' + str(pg_cnt)

In [3]:
result_set = []

while True:

        response = requests.get(url, headers=headers).text
        soup = bs(response,  'html.parser')
    
        a = soup.find_all('li', class_='result-row') 
        
        if len(a) == 0:
                break
        
        else: 
                #loop through returned results
                for item in a:
                        
                        #set empty variables
                        mini_list = []
                        mini_list.append(pg_num)

                        #error handling
                        try:
                                #identify and return title of listing
                                listing = item.find('a', class_='result-title').text
                                mini_list.append(listing)
                        except: 
                                mini_list.append('no name')
        
                        try: 
                                #identify and return price of listing
                                price = item.find('span', class_='result-price').text
                                mini_list.append(price)                                
                        except: 
                                mini_list.append('$0.00')
                                
                        try:
                                #identify and return url of listing                            
                                item_url = item.find('a', class_='result-title hdrlnk')['href']
                                mini_list.append(item_url)
                        except: 
                                mini_list.append('noURL')
                                
                        result_set.append(mini_list) 
                
                #iterate through pages
                pg_cnt += per_pg
                pg_num += 1
                url = 'https://toronto.craigslist.org/search/sss?query='+ str(query) + '&sort=rel&s=' + str(pg_cnt)

In [4]:
#establish dataframe from result set
query_df = pd.DataFrame(result_set)

In [5]:
query_df.head(5)

,0,1,2,3
0,1,Lincoln Idealarc 250 welder (Reduced.),$100,https://toronto.craigslist.org/tor/tls/d/etobi...
1,1,Alternator / Voltage tester,$50,https://toronto.craigslist.org/tor/tls/d/etobi...
2,1,Kitchen tools - New,$5,https://toronto.craigslist.org/drh/hsh/d/whitb...
3,1,2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATT...,$75,https://toronto.craigslist.org/tor/tls/d/2-new...
4,1,Freud Diablo Blades,$100,https://toronto.craigslist.org/tor/tls/d/scarb...


In [6]:
query_df.columns = ["page", "item", "price", "url"]
query_df.head(5)

,page,item,price,url
0,1,Lincoln Idealarc 250 welder (Reduced.),$100,https://toronto.craigslist.org/tor/tls/d/etobi...
1,1,Alternator / Voltage tester,$50,https://toronto.craigslist.org/tor/tls/d/etobi...
2,1,Kitchen tools - New,$5,https://toronto.craigslist.org/drh/hsh/d/whitb...
3,1,2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATT...,$75,https://toronto.craigslist.org/tor/tls/d/2-new...
4,1,Freud Diablo Blades,$100,https://toronto.craigslist.org/tor/tls/d/scarb...


In [7]:
# Identify incomplete rows
query_df.dtypes

page      int64
item     object
price    object
url      object
dtype: object

In [8]:
# Delete extraneous column
del query_df['page']
#sub_df = query_df.iloc['item', 'price', 'url']
query_df.head()

,item,price,url
0,Lincoln Idealarc 250 welder (Reduced.),$100,https://toronto.craigslist.org/tor/tls/d/etobi...
1,Alternator / Voltage tester,$50,https://toronto.craigslist.org/tor/tls/d/etobi...
2,Kitchen tools - New,$5,https://toronto.craigslist.org/drh/hsh/d/whitb...
3,2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATT...,$75,https://toronto.craigslist.org/tor/tls/d/2-new...
4,Freud Diablo Blades,$100,https://toronto.craigslist.org/tor/tls/d/scarb...


In [9]:
#to_drop = ['electrician']
#new_df[~new_df['item'].isin(to_drop)]
#useless_items = ['water filter', 'awesome', 'jewelry', 'camera', 'estate', 'auction', "men's"]
dropped_df = query_df[~query_df['item'].str.contains(
    "water filter|awesome|jewelry|camera|estate|auction|men's|bed|kiln|golf|rake|video|meat|hair|lexus|extensions|butcher|auto|crimper|service|home|figurine|app|phones|facial|symphony|teeth", flags=re.IGNORECASE)]
dropped_df.head()

,item,price,url
0,Lincoln Idealarc 250 welder (Reduced.),$100,https://toronto.craigslist.org/tor/tls/d/etobi...
1,Alternator / Voltage tester,$50,https://toronto.craigslist.org/tor/tls/d/etobi...
2,Kitchen tools - New,$5,https://toronto.craigslist.org/drh/hsh/d/whitb...
3,2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATT...,$75,https://toronto.craigslist.org/tor/tls/d/2-new...
4,Freud Diablo Blades,$100,https://toronto.craigslist.org/tor/tls/d/scarb...


In [10]:
# Drop all rows with missing information
dropped_df = dropped_df.dropna(how='any')
#dropped_df.tail()
#dropped_df.count()
#dropped_df.reset_index()

In [11]:
# remove all dollar signs
dropped_df['price'] = [x.strip('$') for x in dropped_df['price']]
dropped_df.head()

,item,price,url
0,Lincoln Idealarc 250 welder (Reduced.),100,https://toronto.craigslist.org/tor/tls/d/etobi...
1,Alternator / Voltage tester,50,https://toronto.craigslist.org/tor/tls/d/etobi...
2,Kitchen tools - New,5,https://toronto.craigslist.org/drh/hsh/d/whitb...
3,2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATT...,75,https://toronto.craigslist.org/tor/tls/d/2-new...
4,Freud Diablo Blades,100,https://toronto.craigslist.org/tor/tls/d/scarb...


In [12]:
dropped_df_2 = dropped_df[~dropped_df['price'].isin(['0.00'])]
dropped_df_2.tail()
#dropped_df.reset_index()

,item,price,url
1097,Brand New Wood Moisture Meter,30,https://toronto.craigslist.org/tor/tld/d/north...
1098,50 Pcs Screwdriver And Bits Kit,25,https://toronto.craigslist.org/tor/tld/d/north...
1099,Brand New Hybrid Polymer industrial air hose,20,https://toronto.craigslist.org/tor/tld/d/north...
1100,BRAND NEW 12PCS SAFETY SET,8,https://toronto.craigslist.org/tor/tld/d/north...
1101,"Brand New King Canada 10"" Portable Worksite Ta...",250,https://toronto.craigslist.org/tor/tld/d/north...


In [13]:
#dropped_df = dropped_df["price"].sum()
#dropped_df

In [14]:
# output to csv
dropped_df_2.to_csv('csv_outputs/craigslist_' + query + '.csv')

In [15]:
#print('-----Done-------')

In [16]:
# establish mongodb location
conn = "mongodb://localhost:27017"

# collect to mongodb database
client = pymongo.MongoClient(conn)

In [17]:
# establish database name
db = client.toolsdb

In [18]:
# establish collection
tools = db.tools

In [19]:
# call the 'to_dict' function on the new_df dataframe 
# pass the argument'records' to translate each row of the df into a dictionary
data = dropped_df_2.to_dict("records")

In [20]:
data

[{'item': 'Lincoln Idealarc 250 welder (Reduced.)',
  'price': '100',
  'url': 'https://toronto.craigslist.org/tor/tls/d/etobicoke-lincoln-idealarc-250-welder/6789038007.html'},
 {'item': 'Alternator / Voltage tester',
  'price': '50',
  'url': 'https://toronto.craigslist.org/tor/tls/d/etobicoke-alternator-voltage-tester/6789040156.html'},
 {'item': 'Kitchen tools - New',
  'price': '5',
  'url': 'https://toronto.craigslist.org/drh/hsh/d/whitby-kitchen-tools-new/6790540932.html'},
 {'item': '2 NEW WATER PRESSURE SAFETY RELIEF VALVES WATTS 3/4" 174A, 100 PSI',
  'price': '75',
  'url': 'https://toronto.craigslist.org/tor/tls/d/2-new-water-pressure-safety-relief/6805048550.html'},
 {'item': 'Freud Diablo Blades',
  'price': '100',
  'url': 'https://toronto.craigslist.org/tor/tls/d/scarborough-freud-diablo-blades/6805041488.html'},
 {'item': 'ConairPRO Dog 2-in-1 Clipper/Trimmer Kit',
  'price': '40',
  'url': 'https://toronto.craigslist.org/mss/tls/d/conairpro-dog-2-in-1-clipper-trimmer-

In [22]:
# insert list into mongodb
tools.insert_many(data)

In [23]:
# try to find data in database
tools.find()